In [11]:
## Common Import
import time, os, io, shutil
from threading import Thread

## import for screenshot
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## import for google drive 
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from googleapiclient.http import MediaIoBaseDownload

## import for google sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials

## import for video edit
from PIL import Image
import cv2
import numpy as np
from moviepy.editor import *

In [12]:
class App():
    def __init__(self):
        self.SCOPES_SHEET = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets', "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
        self.creds_SHEET = None
        self .client = None
        self.SCOPES_DRIVE = ['https://www.googleapis.com/auth/drive']
        self.creds_DRIVE = None
        self.service = None
        
        self.shotFlag, self.accessFlag=0,0
        self.serviceLocation, self.clientLocation= "", ""
        self.sheetName, self.shotFID, self.vidFID = "", "", ""
        self.options = webdriver.ChromeOptions()
        self.options.add_experimental_option("useAutomationExtension", False)
        self.options.add_experimental_option("excludeSwitches",["enable-automation"])
        # options.add_argument('--headless')
        self.driver = ""
        self.n= None
        self.sheet=None
        self.threadFlag= "N"
        self.recordFlag= True 
        self.home()
        

    def home(self):
            # try:
            while self.accessFlag!=1:
                print("WARNING: Please Provide correct information otherwise code will close automatically due to error")
                self.serviceLocation = "service_secret.json" #input("Enter PATH of Service secret Json file: ")  # "service_secret.json"
                self.clientLocation = "client_secret.json" #input("Enter PATH of Client secret Json file: ")    # "client_secrets.json"
                self.threadFlag = input("Enter (Y/N) to thread part of video Editing: ")
                self.verify()
            if self.accessFlag == 1:
                del self.accessFlag
                self.Continue()
            # except Exception  as e:
            #     print(e)
            #     input()
        
    def verify(self):
        error = ""
        try:
            self.creds_SHEET = ServiceAccountCredentials.from_json_keyfile_name(self.serviceLocation, self.SCOPES_SHEET)
            self.client = gspread.authorize(self.creds_SHEET)
        except:
            error += "client"
        try:
            self.creds_DRIVE = Credentials.from_authorized_user_file(self.clientLocation, self.SCOPES_DRIVE)
            self.service = build('drive', 'v3', credentials=self.creds_DRIVE)
        except:
            error += "service"
        
        if error == "":
            self.accessFlag=1
        if "client" in error or "service" in error:
            print("Error in ", end="")
            if "client" in error:
                print("Client Path ", end='')
            if "client" in error and "service" in error:
                print("and ", end='')
            if "service" in error:
                print("Client Path", end='')
            print('.\n\n')
        del error
        
        

    def Continue(self):
            self.sheetName = ""   #input("Enter Google Sheet Name: ")
            self.vidFID = "" #input("Enter Google Drive Video Folder Id: ")
            self.sheet = self.client.open(self.sheetName).sheet1
            data = self.sheet.get_all_records()
            self.n= len(data)
            a=1
            Openflag =0
            zip_code_updated = False
            self.driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=self.options)
            self.driver.maximize_window()
            for i in data:
                a+=1
                count= 0
                StartFlag = (i['Link'] != "" and i["Video 1"] != "" and i["Video 2"] == "")
                flag2 = i["Video 3"] == ""
                print(i)
                if StartFlag:
                    if "www.amazon.com" in i['Link']:
                        Openflag +=1
                    if Openflag==1 and not zip_code_updated :
                        self.driver.get("https://www.amazon.com/")
                        try:
                            time.sleep(1)
                            self.driver.find_element("xpath", '//*[@id="nav-global-location-popover-link"]').click()
                            time.sleep(2)
                            self.driver.find_element("xpath", '//*[@id="GLUXZipUpdateInput"]').send_keys("10001")
                            time.sleep(1)
                            self.driver.find_element("xpath", '//*[@id="GLUXZipUpdate"]/span').click()
                            time.sleep(1)
                        except:
                            self.driver.get("https://www.amazon.com/")
                            time.sleep(1)
                            self.driver.find_element("xpath", '//*[@id="nav-global-location-popover-link"]').click()
                            time.sleep(2)
                            self.driver.find_element("xpath", '//*[@id="GLUXZipUpdateInput"]').send_keys("10001")
                            time.sleep(1)
                            self.driver.find_element("xpath", '//*[@id="GLUXZipUpdate"]/span').click()
                            time.sleep(1)
                        self.driver.get("https://www.amazon.com/")
                        zip_code_updated = True
                        Openflag=False
                    fileId = i["Video 1"].split("/")[-2]
                    request = self.service.files().get_media(fileId=fileId)
                    file = io.BytesIO()
                    downloader = MediaIoBaseDownload(file, request)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                        print(F'Download {int(status.progress() * 100)}.')
                    file.seek(0)
                    with open(f"{fileId}.mp4", 'wb') as f:
                        shutil.copyfileobj(file, f)
                    self.BGRecorder(i, data[-1])
                    print("Before calling VideoEditor for Keyword:", i["Keyword"])
                    if "N" in self.threadFlag.upper():
                        self.VideoEditor(i["Keyword"], f"{fileId}.mp4")
                        print("After calling VideoEditor for Keyword:", i["Keyword"])
                    else:
                        x=Thread(target=self.VideoEditor, args=(i["Keyword"], f"{fileId}.mp4",), daemon=True)
                        x.start()
                        print("Thread started for Keyword:", i["Keyword"])
                    mergeFlag = False
                    if flag2 == False:
                        self.mergedVideo(i["Keyword"],i["Video 3"])
                        mergeFlag = True
                    self.creds_SHEET = ServiceAccountCredentials.from_json_keyfile_name(self.serviceLocation, self.SCOPES_SHEET)
                    self.client = gspread.authorize(self.creds_SHEET)
                    self.creds_DRIVE = Credentials.from_authorized_user_file(self.clientLocation, self.SCOPES_DRIVE)
                    self.service = build('drive', 'v3', credentials=self.creds_DRIVE)
                    print(f"Uploading Data in Google Drive and Updating Google Sheet for keyword {i['Keyword']}.")
                    link = self.upload(i["Keyword"], mergeFlag, self.vidFID)
                    try:
                        os.unlink(f"{fileId}.mp4")
                        os.remove(f"{fileId}.mp4")
                    except:
                        pass
                    self.updateSheet(i, link, a)
                else:
                    self.n-=1
            self.driver.close()
            

    def BGRecorder(self, i, lastval):
            self.recordFlag = True
            H = self.driver.execute_script("return window.innerHeight;")
            W = self.driver.execute_script("return window.innerWidth;")
            SCREEN_SIZE = (W,H)
            x=Thread(target=self.record, args=(i["Keyword"],SCREEN_SIZE,), daemon=True)
            x.start()
            if "www.amazon.com" in i["Link"]:
                print("Navigating to Amazon link:", i["Link"])
                x = Thread(target=self.record, args=(i["Keyword"], SCREEN_SIZE,), daemon=True)
                x.start()
                if i["Video Type"] == "SEARCH" or i["Video Type"] == "BOTH":
                    self.driver.get("https://www.amazon.com/")
                    keys = ""
                    for key in i["Keyword"]:  # <-- Corrected the variable name here
                        keys += key
                        print("Key:", key)
                        try:
                            search_box = self.driver.find_element("xpath", '//*[@id="twotabsearchtextbox"]')
                            search_box.send_keys(key)
                        except:
                            print("Search box not found. Exiting search.")
                            break
                        time.sleep(2)
                    else:
                        search_submit_button = self.driver.find_element("xpath", '//*[@id="nav-search-submit-button"]')
                        search_submit_button.click()
                        time.sleep(1)
                elif i["Video Type"] == "SCROLL":
                    # Perform the scroll action
                    if i["Video Type"] == "BOTH":
                        pass
                    else:
                        self.driver.get(i["Link"])
                    time.sleep(2)
                    # code for scroll
                    height = round(self.driver.execute_script('return document.body.scrollHeight'))
                    for scrol in range(0, height, 20):
                        self.driver.execute_script(f"window.scrollTo(0,{scrol})")
                        totalScrolledHeight = self.driver.execute_script("return window.pageYOffset + window.innerHeight")
                        pauseHeight = self.driver.execute_script("""return $("[id*='Footer']").offsetTop - window.innerHeight""")
                        if totalScrolledHeight >= pauseHeight:
                            break

            elif "drive.google.com/" in i["Link"]:
                x=Thread(target=self.record, args=(i["Keyword"],SCREEN_SIZE,), daemon=True)
                x.start()
            try:
                self.driver.get(i["Link"])
                print("Successfully navigated to Amazon link.")
                actions = ActionChains(self.driver)
                d = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "ndfHFb-c4YZDc-q77wGc")))
                time.sleep(4)
                page = self.driver.execute_script("return document.getElementsByClassName('ndfHFb-c4YZDc-cYSp0e-DARUcf').length")
                height = self.driver.execute_script("return document.getElementsByClassName('ndfHFb-c4YZDc-cYSp0e-DARUcf')[0].clientHeight")
                total= page*height
                ele = self.driver.find_element(By.TAG_NAME, "body")
                actions.click(ele)
                for i in range(0, total, 40):
                    actions.key_down(Keys.ARROW_DOWN).key_up(Keys.ARROW_DOWN).perform()
                    time.sleep(0.5)
            except Exception as e:
                print("Error occurred during Amazon actions:", e)

            else:
                if i["Video Type"] == "STILL" or i["Video Type"] == "BOTH":
                    self.driver.get(i["Link"])
                    time.sleep(2)
                if i["Video Type"] == "SCROLL" or  i["Video Type"] == "BOTH":
                    if i["Video Type"] == "BOTH":
                        pass
                    else:
                        self.driver.get(i["Link"])
                    time.sleep(2)
                    # code for scroll
                    height = round(self.driver.execute_script('return document.body.scrollHeight'))
                    for scrol in range(0,height, 20):
                        self.driver.execute_script(f"window.scrollTo(0,{scrol})")
                        totalScrolledHeight = self.driver.execute_script("return window.pageYOffset + window.innerHeight")
                        try:
                            pauseHeight= self.driver.execute_script("""return $("[id*='Footer']").offsetTop - window.innerHeight""")
                        except:
                            pauseHeight = totalScrolledHeight
                        if totalScrolledHeight >= pauseHeight:
                            break

            self.recordFlag=False
            self.shotFlag+=1
            del i,lastval
    
        
    def overlayPNG(self, imgBack, imgFront, pos=[0, 0]):
        hf, wf, cf = imgFront.shape
        hb, wb, cb = imgBack.shape
        *_, mask = cv2.split(imgFront)
        maskBGRA = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGRA)
        maskBGR = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        imgRGBA = cv2.bitwise_and(imgFront, maskBGRA)
        imgRGB = cv2.cvtColor(imgRGBA, cv2.COLOR_BGRA2BGR)
        imgMaskFull = np.zeros((hb, wb, cb), np.uint8)
        imgMaskFull[pos[1]:hf + pos[1], pos[0]:wf + pos[0], :] = imgRGB
        imgMaskFull2 = np.ones((hb, wb, cb), np.uint8) * 255
        maskBGRInv = cv2.bitwise_not(maskBGR)
        imgMaskFull2[pos[1]:hf + pos[1], pos[0]:wf + pos[0], :] = maskBGRInv
        imgBack = cv2.bitwise_and(imgBack, imgMaskFull2)
        imgBack = cv2.bitwise_or(imgBack, imgMaskFull)
        del imgMaskFull2, hf, wf, cf, hb, wb, cb, mask, maskBGRA, maskBGR, imgRGBA, imgRGB, imgMaskFull, maskBGRInv
        return imgBack

    def VideoEditor(self, Keyword, VideoPATH, thresh=0.2): 
        clip = VideoFileClip(VideoPATH)
        clip.audio.write_audiofile(f"{Keyword}temp.mp3", logger= None)#, verbose=False)
        duration = clip.duration
        fps = clip.fps
        cap = cv2.VideoCapture(VideoPATH)
        size = (int(cap.get(3)), int(cap.get(4)))
        frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # print(size, frames, fps, duration)
        out = cv2.VideoWriter(f'{Keyword}temp.mp4',cv2.VideoWriter_fourcc(*'MP4V'), fps, size)
        # imgBg = cv2.imread(f"{Keyword}.png")
        bg = cv2.VideoCapture(f"{Keyword}_Bg.mp4")
        n=1
        print(f"{Keyword} video editing start")
        while True:
            if n!=frames:
                success, frame = cap.read()
                sucbg, bgFra = bg.read()
                if sucbg == True or n == 1:
                    imgBg = bgFra
                else:
                    pass
                if n==1:
                    tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   # gray image
                    circles_img = cv2.HoughCircles(tmp,cv2.HOUGH_GRADIENT,1,20,
                                                param1=50,param2=30,minRadius=100,maxRadius=110)
                    circles_img = np.uint16(np.around(circles_img))
                    i = circles_img[0,:][0]
                mask = np.copy(frame)
                imgBg = cv2.resize(imgBg, mask.shape[1::-1], interpolation = cv2.INTER_CUBIC)
                pointa = i[:2]-i[-1]
                pointb = i[:2]+i[-1]
                cv2.circle(mask,(i[0],i[1]),i[2]+49,(0, 0, 0), thickness=100)
                mask[:,:pointa[0]] = 0   # left side of Image
                mask[:,pointb[0]:] = 0   # right side of Image
                mask[:pointa[1],:] = 0   # above side of Image
                mask[pointb[1]:,:] = 0   # bottom side of Image
                tmp = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                _, alpha = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY)
                b, g, r = cv2.split(mask)
                rgba = [b, g, r, alpha]
                dst = cv2.merge(rgba, 4)
                final = self.overlayPNG(imgBg, dst, [0, 0])
                out.write(final)
                n+=1
            else:
                break

        del i, final, dst, _, alpha, tmp, mask, pointa, pointb, circles_img, frame, success
        out.release()
        del out
        video = VideoFileClip(f"{Keyword}temp.mp4", audio=False)
        audio = AudioFileClip(f"{Keyword}temp.mp3")
        final_video = concatenate_videoclips([video], method='compose')
        final_video.audio = audio
        print(f"{Keyword} video editing 60% complete.")
        final_video.write_videofile(f'{Keyword}.mp4')#, logger= None, verbose=False)
        video.close()
        audio.close()
        final_video.close()
        del final_video, audio, video
        os.remove(f"{Keyword}temp.mp3")
        os.remove(f"{Keyword}temp.mp4")
        self.n-=1
        print(f"{Keyword} video editing 100% complete.")
        

  ### Update Google sheet code
    def updateSheet(self, i,videoLink, index):
        insertRow = [i["Video 1"], i["Keyword"], i["Link"], videoLink, i["Video 3"], i["Video Type"]]
        self.sheet.delete_row(index)
        self.sheet.insert_row(insertRow, index)
        del videoLink, index, insertRow
    
   

    def record(self, keyword, SCREEN_SIZE):
        out = cv2.VideoWriter(f"{keyword}_Bg.mp4",cv2.VideoWriter_fourcc(*'MP4V'), 6, SCREEN_SIZE)
        def shot(data):
            nonlocal out
            img = Image.open(io.BytesIO(data))
            frame = np.asarray(img)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            out.write(frame)
            return True
        
        threads=[]
        while self.recordFlag:
            data= self.driver.get_screenshot_as_png()
            y= Thread(target=shot, args=(data,), daemon=True)
            y.start()
            threads.append(y)
        for thread in threads:
            thread.join()
        out.release()
        
        
     ### upload file code
    def upload(self,Keyword, flag, videoFolderID):
        time.sleep(1)
        if flag:
            Keyword = "Full_" + Keyword
        videoMetadata = {'name': f'{Keyword}.mp4'}
        videoMedia = MediaFileUpload(f'{Keyword}.mp4',mimetype='video/mp4')
        
        
        # Retry upload for a maximum of 3 times
        retry_count = 0
        while retry_count < 3:
            try:
                videoId = self.service.files().create(body=videoMetadata, media_body=videoMedia,fields='id, parents').execute()
                video = self.service.files().update(fileId=videoId["id"], addParents=videoFolderID,removeParents=videoId["parents"][0], fields='id').execute()
                videoLink = f"https://drive.google.com/file/d/{video['id']}/"
                break
            except Exception as e:
                print(f"Upload failed: {e}")
                retry_count += 1
                print(f"Retrying upload ({retry_count} of 3 attempts)...")
                time.sleep(5)  # Wait for a few seconds before retrying
                
        else:
            # If upload fails after 3 attempts, raise an error
            raise Exception("Upload failed after 3 attempts. Please check your network connection and try again later.")

         
        # Clean up and return the video link
        del videoMetadata, videoMedia, videoId, video
        os.remove(f"{Keyword}.mp4")
        if flag:
            Keyword = Keyword.replace("Full_", "") 
            os.remove(f"{Keyword}.mp4")
        return videoLink
        
    



    def mergedVideo(self, Keyword, clipname):
        clip = VideoFileClip(f"{Keyword}.mp4")
        clips= [clip]
        if "https://drive.google.com/" not in clipname:
            clip = VideoFileClip(clipNname)
        else:
            fileId = clipname.split("/")[-2]
            request = self.service.files().get_media(fileId=fileId)
            file = io.BytesIO()
            downloader = MediaIoBaseDownload(file, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
            file.seek(0)
            with open(f"{fileId}.mp4", 'wb') as f:
                shutil.copyfileobj(file, f)
            clip = VideoFileClip(f"{fileId}.mp4")
            
        # Insert the second video at the beginning of the clips list
        clips.insert(0, clip)    
        #clips.append(clip)
        final = concatenate_videoclips(clips)
        final.write_videofile(f"Full_{Keyword}.mp4")
        for i in clips:
            i.close()
        try:
            os.unlink(f"{Keyword}_bg.mp4")
            os.remove(f"{Keyword}_bg.mp4")
        except:
            pass
    

# video = # Final video = # based video = 
a = App()

Enter (Y/N) to thread part of video Editing: N
{'Video 1': 'https://drive.google.com/file/d/1LvCm1uoSeXNv7IywnK8IXyeErDveLDaa/view?usp=sharing', 'Keyword': 'Tactacam', 'Link': 'www.amazon.com/s?k=Tactacam', 'Video 2': 'https://drive.google.com/file/d/1fDae_H7AZBpxT1xZs9v0iDQE-qaM1fQ9/', 'Video 3': 'https://drive.google.com/file/d/1poS_QM0nFyyxM6pdahsYQ_Izk03YI620/view?usp=sharing', 'Video Type': 'SEARCH'}
{'Video 1': 'https://drive.google.com/file/d/1LvCm1uoSeXNv7IywnK8IXyeErDveLDaa/view?usp=sharing', 'Keyword': 'Plantronics', 'Link': 'www.amazon.com/s?k=Plantronics', 'Video 2': 'https://drive.google.com/file/d/1ng_vIguPqBzkFOqg_u_2T-0jJsIilj3K/', 'Video 3': 'https://drive.google.com/file/d/1poS_QM0nFyyxM6pdahsYQ_Izk03YI620/view?usp=sharing', 'Video Type': 'SEARCH'}
{'Video 1': 'https://drive.google.com/file/d/1LvCm1uoSeXNv7IywnK8IXyeErDveLDaa/view?usp=sharing', 'Keyword': 'swiftmaps europe', 'Link': 'www.amazon.com/s?k=swiftmaps+europe', 'Video 2': 'https://drive.google.com/file/d/1Pm

RefreshError: ('unauthorized_client: Unauthorized', {'error': 'unauthorized_client', 'error_description': 'Unauthorized'})